In [136]:
import csv, datetime
import glob

# データファイル一覧の読み込み
files=glob.glob("./data/*.csv")

In [323]:
# 本番用test
file_name = './data/33_sample.csv'
file_name_list = file_name.split('/')
result_dir_name = './result/'
result_file_name = result_dir_name + file_name_list[2]
# './result/33_sample.csv'
assert result_file_name ==  './result/33_sample.csv'
# ファイルのread
file = read_csv(file_name)

# ファイルの分割、書き込み
separate_file = separate_row(file, '。')
print('ファイル名： '+ file_name)
print('分割前： ' + str(len(file)))
print('分割後： ' + str(len(separate_file)))

# タイムスタンプの更新
timestamp_row = []
for row in separate_file:
    timestamp_row.append(row[0])
    
next_times = get_next_times(timestamp_row)
next_times2 = get_next_times2(separate_file)

print(len(next_times2), len(next_times), next_times2[0])
vtt_times = get_vtt_times2(next_times2)
assert(len(separate_file)==len(vtt_times))

# ファイルの書き込み
with open(result_file_name, 'w')as o:
    writer = csv.writer(o, delimiter=',')
    for i, row in enumerate(separate_file):
        writer.writerow([vtt_times[i], row[1], row[2]])


ファイル名： ./data/33_sample.csv
分割前： 76
分割後： 179
total179
179 179 00:00:05.000


In [140]:
def read_csv(file_name):
    """ [row, row, row]の形式の２次元リストを返す"""
    file = []
    with open(file_name) as f:
        for row in csv.reader(f):
            file.append(row)
    return file

In [137]:
# 一括で変換（en, jp２ファイルずつ作成）

for file_name in files:
    f_split = file_name.split('/')
    result_en = './result/en/'
    
    # result_jp = './result/jp/'

    out_en = result_en + f_split[2]
    # out_jp = result_jp + f_split[2]
    
    file = []
    with open(file_name) as f:
        print(file_name)
        for row in csv.reader(f):
            file.append(row)
    write(file, out_en, '.')
    # 日本語を修正したい場合コメントを外すこと
    # write_jp(file, out_jp, '。')

./data/33_島田_20200820.csv
./data/37_林_20200820_edited.csv
./data/41_川田_20200819_edited_日本語訳追加.csv
./data/42_松田_20200820_edited.csv
./data/34_境野_20200817_edited_日本語訳追加.csv
./data/39_加賀_20200819_edited.csv
./data/49_宮川_20200819_edited.csv


In [25]:
# enを読み込みファイルの日付を修正
csv_file = './result/en/37_林_20200820_edited.csv'
en = []
with open(csv_file) as f:
    for row in csv.reader(f):
        en.append(row[0])


In [303]:
def get_next_times(en):
    """推定字幕表示時間を抽出する
    TODO: セグメント内の文字数を算出して、割合とする
    """
    first_date = datetime.datetime.strptime(en[0], '%Y/%m/%d %H:%M')
    same_minitues_count = 0
    check_date = first_date
    new_times = []
    check_total = 0
    for e in en:
        current_date = datetime.datetime.strptime(e, '%Y/%m/%d %H:%M')
        if current_date == check_date:
            same_minitues_count += 1
        else:
            # 日付変更
            date = current_date - first_date
            mm = get_h_m_s(date)
            #? length = 60 // same_minitues_count
            check_total += same_minitues_count
            times = get_times(mm, same_minitues_count)
            #print(times,same_minitues_count)
            assert (len(times)==same_minitues_count)
            check_date = current_date
            same_minitues_count = 1
            new_times.extend(times)
    else:
        #print(same_minitues_count)
        date = current_date - first_date
        mm = get_h_m_s(date)
        check_total += same_minitues_count
        times = get_times(mm, same_minitues_count)
        #print(new_times[-1], times[-1])
        if new_times[-1] != times[-1]:
            new_times.extend(times)
    
    return new_times

In [320]:
def get_next_times2(file):
    """推定字幕表示時間を抽出する
    TODO: セグメント内の文字数を算出して、割合とする
    """
    first_date = datetime.datetime.strptime(file[1][0], '%Y/%m/%d %H:%M')
    same_minitutes_count = 0
    same_minitutes_word_num = [] # 同一時間帯の文字数一覧
    check_date = first_date
    new_times = []
    check_total = 0
    for row in file:
        current_date = datetime.datetime.strptime(row[0], '%Y/%m/%d %H:%M')
        word_num = len(row[2])
        if current_date == check_date:
            same_minitutes_count += 1
            #print(word_num , same_minitutes_word_num, type(word_num), type(same_minitutes_word_num))
            same_minitutes_word_num.append(word_num)
        else:
            # 日付変更
            #print(str(current_date), str(first_date))
            date = current_date - first_date - datetime.timedelta(minutes=1)
            #print('date' + str(date))
            mm = get_h_m_s(date)
            #print(mm)
            check_total += same_minitutes_count
            # 値の算出
            times = get_times2(mm, same_minitutes_count, same_minitutes_word_num)
            #print(times)
            assert (len(times)==same_minitutes_count)
            # 値の更新
            check_date = current_date
            same_minitutes_count = 1
            same_minitutes_word_num = [word_num]
            new_times.extend(times)
    else:
        #print(len(new_times), len(times))
        #print(same_minitutes_count)
        date = current_date - first_date 
        mm = get_h_m_s(date)
        check_total += same_minitutes_count
        times = get_times2(mm, same_minitutes_count, same_minitutes_word_num)
        #print(times)
        #print(new_times[-1], times[-1])
        if new_times[-1] != times[-1]:
            new_times.extend(times)
    print('total'+ str(check_total))
    return new_times

In [292]:
a=[]
a.append(1)
a.append(2)
a
dt2 = datetime.timedelta(minutes=1)

In [50]:
def get_vtt_times(new_times):
    vtt_times = []
    for i, new in enumerate(new_times):
        try:
            vtt_time = new + ' --> ' + new_times[i+1]
        except:
            next_minites = get_next_minites(new)
            vtt_time = new + ' --> ' + next_minites
        vtt_times.append(vtt_time)
    return vtt_times

In [310]:
def get_vtt_times2(new_times):
    vtt_times = []
    for i, new in enumerate(new_times):
        if (i==0):
            vtt_time = '00:00:00.000 --> '+ new
        else:
            try:
                vtt_time = new_times[i-1] + ' --> ' + new
            except:
                next_minites = get_next_minites(new)
                vtt_time = new_times[i-1] + ' --> ' + new
        vtt_times.append(vtt_time)
    return vtt_times

In [311]:
en_files=glob.glob("./result/en/*.csv")
time_dir = './result/time/'
for en_file in en_files:
    #if (en_file == './result/en/39_加賀_20200819_edited.csv' or './result/en/49_宮川_20200819_edited.csv'):
    #    continue
    print(en_file)
    en = []
    
    with open(en_file) as f:
        for row in csv.reader(f):
            en.append(row[0])
        print(len(en))        
        output = time_dir + en_file.split('/')[3]
        next_times = get_next_times(en)
        print(len(next_times))
        vtt_times = get_vtt_times(next_times)
        print(len(vtt_times))
        
        assert(len(en)==len(vtt_times))
    with open(output, 'w')as o:
        writer = csv.writer(o, delimiter=',')
        for vtt in vtt_times:
            writer.writerow([vtt])

./result/en/33_島田_20200820.csv
183
183
183
./result/en/37_林_20200820_edited.csv
159
159
159
./result/en/41_川田_20200819_edited_日本語訳追加.csv
194
194
194
./result/en/42_松田_20200820_edited.csv
146
146
146
./result/en/34_境野_20200817_edited_日本語訳追加.csv
218
218
218
./result/en/39_加賀_20200819_edited.csv
122
122
122
./result/en/49_宮川_20200819_edited.csv
204
204
204


In [84]:
i = './result/en/33_島田_20200820.csv'
r = './result/time/'
en = []
with open(i) as f:
    for row in csv.reader(f):
        en.append(row[0])
        
output = r + i.split('/')[3]
next_times = get_next_times(en)
vtt_times = get_vtt_times(next_times)
print(output)

./result/time/33_島田_20200820.csv


In [83]:
with open(output, 'w')as o:
    writer = csv.writer(o, delimiter=',')
    for vtt in vtt_times:
        writer.writerow([vtt])

In [55]:
print(output)

./result/time/49_宮川_20200819_edited.csv


In [256]:
#a='00:21:00.000'
print(a[3:5])
#minites = int(a[3:5])+1
#aa = '0'+str(minites) if len(str(minites))==1 else str(minites)
#aa
#a = a[:3] + aa + a[5:] 
#a



get_next_minites('00:21:00.000')

[]


'00:22:00.000'

In [312]:
def get_next_minites(time):
    current_minites = int(time[3:5])+1
    next_minites = '0'+str(current_minites) if len(str(current_minites))==1 else str(current_minites)
    next_time = time[:3] + next_minites + time[5:]
    return next_time

In [92]:
print(files)

['./data/33_島田_20200820.csv', './data/37_林_20200820_edited.csv', './data/41_川田_20200819_edited_日本語訳追加.csv', './data/42_松田_20200820_edited.csv', './data/39_加賀_20200819_edited.csv', './data/49_宮川_20200819_edited.csv']


In [3]:
#　読み込み
file = []
csv_file = "./34.csv"
with open(csv_file) as f:
    for row in csv.reader(f):
        file.append(row)

In [4]:
# datetimeから時間、分を返す
def get_h_m_s(td):
    m, s = divmod(td.seconds, 60)
    h, m = divmod(m, 60)
    if len(str(h))==1:
        h = '0'+str(h)
    else:
        h = str(h)
    if len(str(m))==1:
        m = '0'+str(m)
    else:
        m = str(m)
    return h, m

In [313]:
# vtt用時間を返す
def get_vtt_time(first_date, current_date, length):
    fd = datetime.datetime.strptime(first_date, '%Y/%m/%d %H:%M')
    cd = datetime.datetime.strptime(current_date, '%Y/%m/%d %H:%M')
    d = cd - fd
    mm = get_h_m_s(d)
    times = get_times(mm, length)
        
    return times

In [7]:
# 同一時間から配分してリストを返す
def get_times(mm, length):
    second = 60 // length
    times = []
    #print(type(mm[0]),type(mm[0]),type(second))
    for i in range(length):        
        ss = '0'+str(second*i) if len(str(second*i))==1 else str(second*i)
        time = mm[0] + ':' + mm[1] + ':' + ss +'.000'
        times.append(time)
    return times

In [322]:
# 同一時間から配分してリストを返す
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
def get_times2(mm, length, word_nums):
    percentages = list(map(lambda i:i/sum(word_nums), word_nums))
    time_percentages = list(map(lambda i:i*60, percentages))
    time_percentages = list(map(lambda i:int(Decimal(str(i)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)),time_percentages))
    
    if (sum(time_percentages) != 60):
        time_percentages[-1] = 60 - sum(time_percentages[:-1])
    assert sum(time_percentages) == 60
    total_time = 0
    times = []
    for i, time in enumerate(time_percentages):
        total_time += time
        ss = '0'+str(total_time) if len(str(total_time))==1 else str(total_time)
        if(ss == '60'):
            minitutes = '0'+str(int(mm[1])+1) if len(str(int(mm[1])+1))==1 else str(int(mm[1])+1)
            t = mm[0] + ':' + minitutes + ':00.000'
        else:
            t = mm[0] + ':' + mm[1] + ':' + ss +'.000'
        times.append(t)
    #print(type(mm[0]),type(mm[0]),type(second))
    #for i in range(length):        
    #    ss = '0'+str(second*i) if len(str(second*i))==1 else str(second*i)
    #    time = mm[0] + ':' + mm[1] + ':' + ss +'.000'
    #    times.append(time)

    
    
    return times

In [239]:
a = [1,2,3]
print(a[:-1])
print(6 - sum(a[:-1]))

[1, 2]
3


In [216]:

a = [100,22,33,44,55,61,27,18,9,10]
print(sum(a))
b = list(map(lambda i:i/sum(a), a))
print(b, sum(b))
c = list(map(lambda i:i*60, list(b)))
print(c)
d = list(map(lambda i:int(Decimal(str(i)).quantize(Decimal('0'), rounding=ROUND_HALF_UP)),c))
print(d)
print(sum(d))


379
[0.2638522427440633, 0.05804749340369393, 0.0870712401055409, 0.11609498680738786, 0.14511873350923482, 0.16094986807387862, 0.0712401055408971, 0.047493403693931395, 0.023746701846965697, 0.026385224274406333] 0.9999999999999999
[15.831134564643799, 3.4828496042216357, 5.224274406332454, 6.965699208443271, 8.70712401055409, 9.656992084432718, 4.274406332453826, 2.8496042216358837, 1.4248021108179418, 1.58311345646438]
[16, 3, 5, 7, 9, 10, 4, 3, 1, 2]
60


In [10]:
def write(file, output_file, delimiter):
    with open(output_file, 'w')as output:
        writer = csv.writer(output)
        first_date = file[1][0]
        for row_index, row in enumerate(file):
            if(row_index==0):
                continue
            current_date = row[0]
            sentences = row[3].split(delimiter)
            sentences_length = len(sentences)
            
            # 文字列長ごとに時間を割りふる
            if (len(sentences) == 1):
                new_row = [row[0], row[1], row[2], sentences[0].replace(' "', '')]
                #new_row = [times[0], row[1], row[2], sentences[0].replace(' "', '')]
                writer.writerow(new_row)
            else:
                for index, sentence in enumerate(sentences):
                    if (sentence == '' or sentence == '"' or sentence==' "'):
                        continue
                    new_row = [row[0], row[1], row[2], sentence.replace(' "', '')]
                    #new_row = [times[index], row[1], row[2], sentence.replace(' "', '')]
                    writer.writerow(new_row)

In [123]:
def write_jp(file, output_file, delimiter):
    with open(output_file, 'w')as output:
        writer = csv.writer(output)
        first_date = file[1][0]
        for row_index, row in enumerate(file):
            if(row_index==0):
                continue
            current_date = row[0]
            sentences = row[2].split(delimiter)
            sentences_length = len(sentences)
            
            # 文字列長ごとに時間を割りふる
            if (len(sentences) == 1):
                new_row = [row[0], row[1], sentences[0].replace(' "', ''), row[3]]
                #new_row = [times[0], row[1], row[2], sentences[0].replace(' "', '')]
                writer.writerow(new_row)
            else:
                for index, sentence in enumerate(sentences):
                    if (sentence == '' or sentence == '"' or sentence==' "'):
                        continue
                    new_row = [row[0], row[1], sentence.replace(' "', ''), row[3]]
                    #new_row = [times[index], row[1], row[2], sentence.replace(' "', '')]
                    writer.writerow(new_row)

In [179]:
def separate_row(file, delimiter):
    """
    日本語用
    ファイルから句点ごとに変更したリストを返す
    """
    new_file = []
    first_date = file[1][0] #1行目が項目名の想定
    
    for row_index, row in enumerate(file):
        if(row_index==0):
            continue
        current_date = row[0]
        # delimiteerごとに文章を分割
        sentences = row[2].split(delimiter)
        sentences_length = len(sentences)

        # 分割した文章数ごとの処理
        if (sentences_length == 1):
            new_sentence = check_quote(sentences[0]) + delimiter
            new_row = [row[0], row[1], new_sentence, row[3]]
            new_file.append(new_row)
        else:
            for index, sentence in enumerate(sentences):
                if (sentence == '' or sentence == '"' or sentence==' "'):
                    continue
                new_sentence = check_quote(sentence) + delimiter
                new_row = [row[0], row[1], new_sentence, row[3]]
                new_file.append(new_row)
    return new_file

In [178]:
def check_quote(sentence):
    """Meeting Assistの不要な文字列を削除"""
    if sentence[:1] == '"':
        sentence = sentence[1:]
    if sentence[:2] ==' "':
        sentence = sentence[2:]
    if sentence[-1] == '"':
        sentence = sentence [:-1]
    if sentence[-2:] == ' "':
        sentence = sentence [:-2]
    return sentence

In [175]:

check_quote(' "★★★opening★★★。AI 最新動向及び With/AfterコロナにおけるAI技術の活用と題しまして、"')
check_quote(' "本日ですが、特にAI技術の概念の中で、深層学習と言われるものがあります。それは、AIの今の財産で、AIブームを引き起こしたものでもあるのですが、そちらを中心にですね、特に研究開発のスピードが速い、画像系だったりとか、言語系とか"')

★★★opening★★★。AI 最新動向及び With/AfterコロナにおけるAI技術の活用と題しまして、
本日ですが、特にAI技術の概念の中で、深層学習と言われるものがあります。それは、AIの今の財産で、AIブームを引き起こしたものでもあるのですが、そちらを中心にですね、特に研究開発のスピードが速い、画像系だったりとか、言語系とか


In [11]:
output_file = './34_out.csv'
delimiter = '.'
write(file, output_file, delimiter)